In [55]:
import pandas as pd
import unicodedata

Foi usada apenas uma biblioteca a mais para realizar a remoção da acentuação das colunas.

In [56]:

def remover_espacos_acentos(texto):
    """
    Remove espaços e acentos de uma string e substitui os espaços por underscores.

    Argumentos:
    texto (str): Texto a ser processado.

    Retorna:
    str: Texto sem espaços e acentos, com espaços substituídos por underscores.
    """
    if isinstance(texto, str):
        texto = texto.strip().replace(' ', '_')
        texto = ''.join(caracter for caracter in unicodedata.normalize('NFKD', texto) if not unicodedata.combining(caracter))
    return texto

def renomear_coluna(df):
    """
    Renomeia as colunas do DataFrame com base nos últimos valores da última linha.

    Argumentos:
    df (DataFrame): DataFrame a ser processado.

    Retorna:
    DataFrame: DataFrame com as colunas renomeadas.
    """
    df = df.copy()
    ultimos_valores = df.iloc[-1]
    for coluna, valor in ultimos_valores.items():
        if coluna != 'ESTUDANTE':
            novo_nome = f"QT_{coluna}_{remover_espacos_acentos(valor)}"
            df = df.rename(columns={coluna: novo_nome})
    return df

def gerar_pontuacao_simulado(df):
    """
    Calcula a pontuação total dos alunos em um simulado.

    Argumentos:
    df (DataFrame): DataFrame contendo as respostas dos alunos.

    Retorna:
    DataFrame: DataFrame com a pontuação total de cada aluno.
    """
    df = df.copy()
    colunas_inteiras = df.select_dtypes(include='int32').columns
    df['QTD_ACERTOS'] = df[colunas_inteiras].sum(axis=1)
    return df

def calcular_acertos_temas(df):
    """
    Calcula a quantidade de acertos por subgrupo de disciplinas.

    Argumentos:
    df (DataFrame): DataFrame contendo as respostas dos alunos.

    Retorna:
    DataFrame: DataFrame com a quantidade de acertos por subgrupo de disciplinas.
    """
    df = df.copy()
    terminacao_subgrupo = {
        'QUIMICA': 'ACERTOS_CN',
        'FISICA': 'ACERTOS_CN',
        'BIOLOGIA': 'ACERTOS_CN',
        'HISTORIA':'ACERTOS_CH',
        'GEOGRAFIA':'ACERTOS_CH',
        'FILOSOFIA':'ACERTOS_CH',
        'SOCIOLOGIA':'ACERTOS_CH',
        'LITERATURA': 'ACERTOS_LG',
        'INTERTEXTO': 'ACERTOS_LG',
        'PORTUGUES': 'ACERTOS_LG',
        'INGLES':'ACERTOS_LG',
        'MATEMATICA': 'ACERTOS_MT'
    }
    subgrupos_somas = {}
    for coluna in df.columns:
        for terminacao, subgrupo in terminacao_subgrupo.items():
            if coluna.endswith(terminacao):
                if subgrupo not in subgrupos_somas:
                    subgrupos_somas[subgrupo] = df[coluna]
                else:
                    subgrupos_somas[subgrupo] += df[coluna]
                break
    df_somas = pd.DataFrame(subgrupos_somas)
    return df_somas

def remover_espacos_colunas(df):
    """
    Remove espaços nas colunas do DataFrame.

    Argumentos:
    df (DataFrame): DataFrame a ser processado.

    Retorna:
    DataFrame: DataFrame com as colunas sem espaços.
    """
    for coluna in df.columns:
        if isinstance(coluna, str):  
            novo_nome = coluna.strip().replace(' ', '')  
            df.rename(columns={coluna: novo_nome}, inplace=True)  
    return df

def extrair_abas_excel(nome_arquivo):
    """
    Extrai as abas de um arquivo Excel e remove espaços nas colunas de cada aba.

    Argumentos:
    nome_arquivo (str): Nome do arquivo Excel.

    Retorna:
    dict: Dicionário onde as chaves são os nomes das abas e os valores são os DataFrames correspondentes.
    """
    xls = pd.ExcelFile(nome_arquivo)
    abas_dict = {}
    for aba_nome in xls.sheet_names:
        df = xls.parse(aba_nome)
        aba_nome = aba_nome.strip()
        abas_dict[aba_nome] = remover_espacos_colunas(df)
    return abas_dict

def verificar_respostas(df):
    """
    Compara as respostas dos alunos com o gabarito e atribui 1 para respostas corretas e 0 para respostas erradas.

    Argumentos:
    df (DataFrame): DataFrame contendo as respostas dos alunos e o gabarito.

    Retorna:
    DataFrame: DataFrame com 1 para respostas corretas e 0 para respostas erradas.
    """
    df = df.copy()
    gabarito = df.iloc[-1]
    for coluna in df.columns:
        if coluna != "ESTUDANTE":
            df[coluna] = (df[coluna] == gabarito[coluna]).astype(int)
    return df.dropna()

# Correção dos simulados

São carregados os simulados e cruzado com a leitura de seus respectivos gabaritos afim de criar uma base que seja possível analisar a quantidade de acertos e também por área do ENEM (CH,CN,Matématica e Lingaugens)

In [57]:
gabaritos_simulado = extrair_abas_excel('data/raw/Gabaritos_AMPs_2023.xlsx')

## Simulado AMP 1.2

### Carregamento do simulado

In [58]:
simulado_1 = pd.read_excel('data/raw/Respostas_AMP_1.2_2023.xlsx')

### Unindo o simulado com o gabarito correspodente

In [59]:
simulado_1_completo = simulado_1.merge(gabaritos_simulado['AMP_1.2'],how='outer')

### Renomeando as colunas 

É adicionado a qual disciplina a questão pertence para assim facilitar o agrupamento posteriormente

In [60]:
simulado_1_parcial = renomear_coluna(simulado_1_completo)

### Verificação das respostas 
É verificado se a alternativa marcada foi de fato a correta para  a questão, se sim é adicionado 1 se não 0.

In [61]:
simulado_1_parcial = verificar_respostas(simulado_1_parcial.loc[:simulado_1_parcial.shape[0]-2])


### Categorização das respostas por área

Por fim é gerado o resultado por área de conhecimento

In [62]:
simulado_1_final = pd.concat([gerar_pontuacao_simulado(simulado_1_parcial), calcular_acertos_temas(simulado_1_parcial)],axis=1)

## Simulado AMP 2.1

In [63]:
simulado_2 = pd.read_excel('data/raw/Respostas_AMP_2.1_2023.xlsx')

In [64]:
simulado_2.drop('Tipo de Prova',axis=1,inplace=True)

In [65]:
simulado_2_completo = simulado_2.merge(gabaritos_simulado['AMP_2.1'],how='outer')

In [66]:
simulado_2_parcial = renomear_coluna(simulado_2_completo)

In [67]:
simulado_2_parcial = verificar_respostas(simulado_2_parcial.loc[:simulado_2_parcial.shape[0]-2])


In [68]:
simulado_2_final = pd.concat([gerar_pontuacao_simulado(simulado_2_parcial), calcular_acertos_temas(simulado_2_parcial)],axis=1)

## Simulado AMP 2.2

In [70]:
simulado_3 = pd.read_excel('data/raw/Respostas_AMP_2.2_2023.xlsx')

In [71]:
simulado_3.drop('Tipo de Prova',axis=1,inplace=True)

In [72]:
simulado_3_completo = simulado_3.merge(gabaritos_simulado['AMP_2.2'],how='outer')

In [73]:
simulado_3_parcial = renomear_coluna(simulado_3_completo)

In [74]:
simulado_3_parcial = verificar_respostas(simulado_3_parcial.loc[:simulado_3_parcial.shape[0]-2])


In [75]:
simulado_3_final = pd.concat([gerar_pontuacao_simulado(simulado_3_parcial), calcular_acertos_temas(simulado_3_parcial)],axis=1)

## Exportando as bases para o BI

In [76]:
simulado_1_final.to_csv('data/serving/simulado_final_1.2.csv',index=False)

In [77]:
simulado_2_final.to_csv('data/serving/simulado_final_2.1.csv',index=False)

In [78]:
simulado_3_final.to_csv('data/serving/simulado_final_2.2.csv',index=False)